In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [1]:
mobile_train = pd.read_csv('train.csv')
mobile_train

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [4]:
y = mobile_train['price_range']
X = mobile_train.drop(['price_range'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [8]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=2)
my_model.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=2, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [10]:
preds = my_model.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

from sklearn.metrics import mean_squared_error
score2 = mean_squared_error(y_valid, preds)
print('MSE:', score2)

score3 = my_model.score(X_valid, y_valid)
print('Score:', score3)

MAE: 0.1711237657815218
MSE: 0.07372066696244936
Score: 0.9427901078981458


In [11]:
#now predict our test dataset
mobile_test= pd.read_csv('test.csv')
#straight to predict
test_data_final = mobile_test.drop(['id'], axis=1)
preds_test = my_model.predict(test_data_final)
mobile_price_result = pd.DataFrame({'id': mobile_test['id'], 'Price_Range_Prediction': preds_test})
mobile_price_result.to_csv('Mobile_Price__Range_Prediction_Result_XGBoost.csv')

In [13]:
#CROSS VALIDATION
#using the 3rd fold with highest score in random forest model
from sklearn.model_selection import KFold

#now we see the score, lets perform CV manually
#this is the problem, im stuck here
kf = KFold(n_splits = 5, shuffle = True)

split_train_index = []
split_valid_index = []

for train_index, valid_index in kf.split(X):
    split_train_index.append(train_index)
    split_valid_index.append(valid_index)
    
#now we take the 3rd fold (indx starts from 0, so 2nd index (ordinal) is the 3rd fold)
training_data_index = split_train_index[2]
valid_data_index = split_train_index[2]

#form the train valid manually baase on fold
X_train = X.iloc[training_data_index]
y_train = y.iloc[training_data_index]
X_valid = X.iloc[valid_data_index]
y_valid = y.iloc[valid_data_index]

In [14]:
my_model2 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=2)
my_model2.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# Preprocessing of training data, fit model 
my_model2.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_model2.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

from sklearn.metrics import mean_squared_error
score2 = mean_squared_error(y_valid, preds)
print('MSE:', score2)

score3 = my_model2.score(X_valid, y_valid)
print('Score:', score3)
#much better score

MAE: 0.0028101042099297046
MSE: 1.8746827505327634e-05
Score: 0.9999851026297024


In [16]:
test_data_final2 = mobile_test.drop(['id'], axis=1)
preds_test2 = my_model2.predict(test_data_final)
mobile_price_result2 = pd.DataFrame({'id': mobile_test['id'], 'Price_Range_Prediction': preds_test2})
mobile_price_result2.to_csv('Mobile_Price__Range_Prediction_Result2_XgBoost.csv')

In [17]:
res2 = pd.read_csv('Mobile_Price__Range_Prediction_Result2.csv', index_col=0)
res2

,id,Price_Range_Prediction
0,1,2.92
1,2,2.96
2,3,2.30
3,4,2.98
4,5,1.02
...,...,...
995,996,1.54
996,997,1.19
997,998,0.79
998,999,1.99


In [18]:
print(res2.head(10).to_markdown())

|    |   id |   Price_Range_Prediction |
|---:|-----:|-------------------------:|
|  0 |    1 |                     2.92 |
|  1 |    2 |                     2.96 |
|  2 |    3 |                     2.3  |
|  3 |    4 |                     2.98 |
|  4 |    5 |                     1.02 |
|  5 |    6 |                     2.99 |
|  6 |    7 |                     3    |
|  7 |    8 |                     0.87 |
|  8 |    9 |                     2.92 |
|  9 |   10 |                     0    |
